# ING Insights


Notebook used to categorize and automatically analyse transations data from ING Bank Account. This account is aggregation transaction information from all accounts in ING Bank as well as Millennium.

Analysis should be used to aggregate per month infomrations infomrations about spending, mainly from accounts `Wspólne`, `Osobiste`, `Millennium`.


## 01 - Importing data

Import data from ING Bank history export. Class used to import the data `TransactionDataLoader` consisting fetching mechanism with data sanitization finctionalities.

In [268]:
from tabulate import tabulate
from pathlib import Path
from data_loader import TransactionDataLoader

csv_import_1 = Path("data/20241220_lista_transkcji_ing_90.csv")
csv_import_2 = Path("data/20241220_lista_transkcji_ing_mill_90.csv")
csv_import_3 = Path("data/Lista_transakcji_nr_0202026071_161224_ING_MILL.csv")

df = TransactionDataLoader(csv_import_3).get_data()

# If Data transakcji is  empty, fill it with Data księgownia
df.loc[df["Data transakcji"].isna(), "Data transakcji"] = df["Data księgowania"]

# For all blocked transatcions fill the Kwota tranakcji with tha blocked ammount.
df.loc[df["Kwota transakcji (waluta rachunku)"].isna(), "Kwota transakcji (waluta rachunku)"] = df["Kwota blokady/zwolnienie blokady"]

print(f"Succesfully imported dataftame with size {df.shape}\n\n", tabulate(df.head(20), headers='keys'))

Succesfully imported dataftame with size (789, 18)

     Data transakcji    Data księgowania    Dane kontrahenta                                    Tytuł                                           Nr rachunku                     Nazwa banku                         Szczegóły    Nr transakcji           Kwota transakcji (waluta rachunku)  Waluta      Kwota blokady/zwolnienie blokady  Waluta.1      Kwota płatności w walucie    Waluta.2  Konto       Bank                    Saldo po transakcji  Waluta.3
--  -----------------  ------------------  --------------------------------------------------  ----------------------------------------------  ------------------------------  ----------------------------------  -----------  --------------------  ------------------------------------  --------  ----------------------------------  ----------  ---------------------------  ----------  ----------  --------------------  ---------------------  ----------
 0  16/12/2024         16/12/2024          #LN#

## 02 - Data filtering and sanitization


Filter only usefull columns for further data analysiss

In [272]:
selected_columns = ['Data transakcji', 'Dane kontrahenta','Tytuł',  'Kwota transakcji (waluta rachunku)', 'Bank']
new_df = df[selected_columns]

print(tabulate(new_df.head(10), headers='keys'))

    Data transakcji    Dane kontrahenta                                    Tytuł                                             Kwota transakcji (waluta rachunku)  Bank
--  -----------------  --------------------------------------------------  ----------------------------------------------  ------------------------------------  --------------------
 0  16/12/2024         #LN#000560803741 SPŁATA RATY KREDYT U               #LN#000560803741 SPŁATA RATY KREDYT U                                       -5109.87  Bank Millennium S.A
 1  16/12/2024         ROSSMANN 36  Krakow POL 2024-12-14                  ROSSMANN 36  Krakow POL 2024-12-14                                           -125.95  Bank Millennium S.A
 2  16/12/2024         SODEL ALBERT ŁUKASZ SADOWA 28 55-00 2 DOBRZYKOWICE  Swoje                                                                         150     Bank Millennium S.A
 3  16/12/2024         PayPro S.A.                                         /OPT/X/////P24-H4R-T4B-H0J        

## 03 - Category Definition



## 04 - Data Analysis



## 05 - Data Presentation


## 06 - Export to Excel file